In [1]:
import torch
import torch.nn as nn
import pandas as pd
import re

In [4]:
# words=open('names.txt','r').read().splitlines()

In [3]:
Wo=open('IndianNames.txt','r').read().splitlines()

In [4]:
words=[re.sub(r'[^a-z ]', ' ', w.lower()) for w in Wo]

In [5]:
chars=sorted(list(set(''.join(words))))

In [6]:
stoi={s:i+1 for i,s in enumerate(chars)}
stoi['.']=0
itos={i:s for s,i in stoi.items()}

In [11]:
# itos

In [12]:
xs,ys=[],[]
for w in words:
    name=['.']+list(w)+['.']
    for x,y in zip(name,name[1:]):
        xs.append(stoi[x])
        ys.append(stoi[y])
xs=torch.tensor(xs)
ys=torch.tensor(ys)

In [13]:
xs

tensor([ 0,  2,  2,  ...,  2, 12,  2])

In [14]:
xs.shape

torch.Size([506379])

In [21]:
class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()

    def forward(self, output, target, num):
        loss=-output[torch.arange(num),target].log().mean()
        return loss

In [22]:
class ExpNormalizeLayer(nn.Module):
    def forward(self, x):
        x = x.exp()  # Apply exponential
        x = x / x.sum(1, keepdim=True)  # Normalize to make sum equal to 1
        return x

In [15]:
X=torch.nn.functional.one_hot(xs,num_classes=28).float()
num=xs.nelement()

In [16]:
Y=nn.functional.one_hot(ys,num_classes=28).float()

In [23]:
model=torch.nn.Sequential(
    torch.nn.Linear(28,28),
    ExpNormalizeLayer()
)
loss_fn=CustomLoss()
optimiser=torch.optim.SGD(model.parameters(),lr=10)
num_examples=ys.nelement()

In [24]:
num_examples

506379

In [7]:
import onnx
from onnx2pytorch import ConvertModel

In [25]:
for i in range(500):
    optimiser.zero_grad()
    # forward pass
    op=model(X)
    loss=loss_fn(op,ys,num)
    # backward pass
    loss.backward()
    optimiser.step()
    print(loss.data)

tensor(3.3779)
tensor(2.8776)
tensor(2.7310)
tensor(2.6520)
tensor(2.5977)
tensor(2.5563)
tensor(2.5232)
tensor(2.4961)
tensor(2.4734)
tensor(2.4542)
tensor(2.4377)
tensor(2.4234)
tensor(2.4109)
tensor(2.3998)
tensor(2.3900)
tensor(2.3811)
tensor(2.3731)
tensor(2.3659)
tensor(2.3593)
tensor(2.3532)
tensor(2.3476)
tensor(2.3424)
tensor(2.3376)
tensor(2.3332)
tensor(2.3290)
tensor(2.3250)
tensor(2.3213)
tensor(2.3179)
tensor(2.3146)
tensor(2.3115)
tensor(2.3085)
tensor(2.3057)
tensor(2.3031)
tensor(2.3005)
tensor(2.2981)
tensor(2.2958)
tensor(2.2935)
tensor(2.2914)
tensor(2.2894)
tensor(2.2874)
tensor(2.2855)
tensor(2.2837)
tensor(2.2820)
tensor(2.2803)
tensor(2.2787)
tensor(2.2771)
tensor(2.2756)
tensor(2.2741)
tensor(2.2727)
tensor(2.2714)
tensor(2.2700)
tensor(2.2687)
tensor(2.2675)
tensor(2.2663)
tensor(2.2651)
tensor(2.2640)
tensor(2.2629)
tensor(2.2618)
tensor(2.2607)
tensor(2.2597)
tensor(2.2587)
tensor(2.2578)
tensor(2.2568)
tensor(2.2559)
tensor(2.2550)
tensor(2.2541)
tensor(2.2

In [1]:
stoi

NameError: name 'stoi' is not defined

In [10]:
onnx_model = onnx.load("test_model.onnx")

pytorch_model = ConvertModel(onnx_model)

In [11]:
for i in range(10):
    out=[]
    ix=0
    while True:
        xin=nn.functional.one_hot(torch.tensor([ix]),num_classes=28).float()
        op=pytorch_model(xin)
        ix=torch.multinomial(op,num_samples=1,replacement=True).item()
        out.append(itos[ix])
        if(ix==0):
            break
    print(''.join(out))

prasmaa.
agogya.
lithashana.
kanthani.
cirahasa.
ikoghasasath.
gruhantjanthadmat.
n.
a.
aharinbmatvanaaramashila.


In [347]:
v=nn.functional.one_hot(torch.tensor([0]),num_classes=28).float()

In [354]:
v

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [193]:
op=model(v)

In [223]:
x=torch.multinomial(op,num_samples=1,replacement=True).item()

In [224]:
print(x,op[0,x].item())

20 0.04079219698905945


In [216]:
op.min()

tensor(0.0012, grad_fn=<MinBackward1>)

In [215]:
op.max()

tensor(0.1378, grad_fn=<MaxBackward1>)

In [196]:
op

tensor([[0.0012, 0.1378, 0.0406, 0.0480, 0.0526, 0.0479, 0.0129, 0.0207, 0.0272,
         0.0186, 0.0755, 0.0923, 0.0491, 0.0791, 0.0356, 0.0124, 0.0160, 0.0025,
         0.0512, 0.0641, 0.0408, 0.0029, 0.0116, 0.0095, 0.0040, 0.0168, 0.0289]],
       grad_fn=<DivBackward0>)

In [169]:
o.item()

2

In [349]:
model.eval()

Sequential(
  (0): Linear(in_features=28, out_features=28, bias=True)
  (1): ExpNormalizeLayer()
)

In [28]:
dummy_input=torch.randn(1,28)

In [29]:
torch.onnx.export(model, dummy_input, 'test_model.onnx', export_params=True, opset_version=11, do_constant_folding=True)